In [ ]:
# organize dataset into a useful structure
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random
# create directories
dataset_home = 'dataset_dogs_vs_cats/'
subdirs = ['train/', 'test/']
for subdir in subdirs:
	# create label subdirectories
	labeldirs = ['dogs/', 'cats/']
	for labldir in labeldirs:
		newdir = dataset_home + subdir + labldir
		makedirs(newdir, exist_ok=True)
# seed random number generator
seed(1)
# define ratio of pictures to use for validation
val_ratio = 0.25
# copy training dataset images into subdirectories
src_directory = 'train/'
for file in listdir(src_directory):
	src = src_directory + '/' + file
	dst_dir = 'train/'
	if random() < val_ratio:
		dst_dir = 'test/'
	if file.startswith('cat'):
		dst = dataset_home + dst_dir + 'cats/'  + file
		copyfile(src, dst)
	elif file.startswith('dog'):
		dst = dataset_home + dst_dir + 'dogs/'  + file
		copyfile(src, dst)

In [ ]:
# vgg16 model used for transfer learning on the dogs and cats dataset
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

# define cnn model
def define_model():
	# load model
	model = VGG16(include_top=False, input_shape=(224, 224, 3))
	# mark loaded layers as not trainable
	for layer in model.layers:
		layer.trainable = False
	# add new classifier layers
	flat1 = Flatten()(model.layers[-1].output)
	class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
	output = Dense(1, activation='sigmoid')(class1)
	# define new model
	model = Model(inputs=model.inputs, outputs=output)
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	return model

# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

# run the test harness for evaluating a model
def run_test_harness():
	# define model
	model = define_model()
	# create data generator
	datagen = ImageDataGenerator(featurewise_center=True)
	# specify imagenet mean values for centering
	datagen.mean = [123.68, 116.779, 103.939]
	# prepare iterator
	train_it = datagen.flow_from_directory('dataset_dogs_vs_cats/train/',
		class_mode='binary', batch_size=64, target_size=(224, 224))
	test_it = datagen.flow_from_directory('dataset_dogs_vs_cats/test/',
		class_mode='binary', batch_size=64, target_size=(224, 224))
	# fit model
	history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
		validation_data=test_it, validation_steps=len(test_it), epochs=10, verbose=1)
	# evaluate model
	_, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	summarize_diagnostics(history)
    model.save('final_model.h5')
# entry point, run the test harness
run_test_harness()

In [3]:
from tensorflow.keras.models import load_model
model = load_model('final_model.h5')
print(model.outputs)

[<tf.Tensor 'dense_8/Identity:0' shape=(None, 1) dtype=float32>]


In [4]:
print(model.inputs)

[<tf.Tensor 'input_3:0' shape=(None, 224, 224, 3) dtype=float32>]


In [1]:
# make a prediction for a new image.
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(224, 224))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 224, 224, 3)
	# center pixel data
	img = img.astype('float32')
	img = img - [123.68, 116.779, 103.939]
	return img

# load an image and predict the class

# load the image
img = load_image('golden-retriever-royalty-free-image-506756303-1560962726.jpg')
# load model
model = load_model('final_model.h5')
# predict the class
result = model.predict(img)
#print(result[0])
if(result[0]==0):
    print('Cat')
else:
    print('Dog')

# entry point, run the example


Dog


In [6]:
model.save('my_model2')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: my_model2\assets


In [7]:
import tensorflow as tf
tf.saved_model.save(model, "my_model3")

INFO:tensorflow:Assets written to: my_model3\assets


In [8]:
loaded = tf.saved_model.load("my_model3")
print(list(loaded.signatures.keys()))

['serving_default']


In [14]:
infer = loaded.signatures["serving_default"]
print(infer.structured_outputs)

{'dense_8': TensorSpec(shape=(None, 1), dtype=tf.float32, name='dense_8')}


In [35]:
! saved_model_cli show --dir pets --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['mobilenetv2_1.00_128_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 128, 128, 3)
      name: serving_default_mobilenetv2_1.00_128_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict
2020-06-01 10:49:59.564034: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_101.dll


In [3]:
! saved_model_cli show --dir custom_sig --tag_set serve --signature_def similarity

The given SavedModel SignatureDef contains the following input(s):
  inputs['image_tensor'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 224, 224, 3)
      name: similarity_image_tensor:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['output_0'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [34]:
! saved_model_cli show --dir my_model3 --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['input_3'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 224, 224, 3)
      name: serving_default_input_3:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_8'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict
2020-06-01 10:46:01.319710: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_101.dll


In [21]:
import numpy as np
img = load_image('golden-retriever-royalty-free-image-506756303-1560962726.jpg')
#img=np.expand_dims(img, axis=0)
print(img.shape)
img = np.expand_dims(img, axis=0)
print(img.shape)
img = img.tolist()


(1, 224, 224, 3)
(1, 1, 224, 224, 3)


In [2]:
import numpy as np
img = load_image('golden-retriever-royalty-free-image-506756303-1560962726.jpg')

In [6]:
img=np.reshape(img, (-1,224,224,3))
img.shape

(1, 224, 224, 3)

In [7]:
np.save('img',img)

In [1]:
! saved_model_cli run --dir my_model3 --tag_set serve --signature_def serving_default --inputs "input_3=img.npy" --outdir model_cli_run_output

In [ ]:
! git clone https://github.com/matterport/Mask_RCNN.git

In [ ]:
import os 
os.chdir('/content/Mask_RCNN')

In [ ]:
! ls

In [ ]:
os.chdir('/content')

In [ ]:
! git clone https://github.com/experiencor/kangaroo.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Saving a Model with Custom Signature

In [1]:
from tensorflow.keras.models import load_model
import tensorflow as tf
model = load_model('final_model.h5')

In [28]:
class CustomSig(tf.Module):
    def __init__(self,model):
        super(CustomSig, self).__init__()
        self.model=model

    @tf.function
    def __call__(self,image_tensor):
        return self.model(image_tensor)     

modul = CustomSig(model)

signatures = {"similarity": modul.__call__.get_concrete_function(tf.TensorSpec(shape=(None,224,224,3), dtype=tf.float32))}

tf.saved_model.save(modul, "custom_sig", signatures=signatures)

INFO:tensorflow:Assets written to: custom_sig\assets


### Listing the signature keys

In [36]:
m = tf.saved_model.load("custom_sig")

In [37]:
list(m.signatures.keys())

['similarity']

In [38]:
tf.__version__

'2.1.0'

### Saving a model with intermediate layer as Output

In [4]:
from tensorflow.keras.models import load_model
import tensorflow as tf
model = load_model('final_model.h5')

In [5]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [14]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
mid_output = model.get_layer('dense_7').output
image_input = model.get_layer('input_3').output
#image_input = Input(shape=(None,224,224,3))

In [17]:
mid_output

<tf.Tensor 'dense_7_1/Identity:0' shape=(None, 128) dtype=float32>

In [15]:
image_input

<tf.Tensor 'input_3_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [18]:
final_model = Model(inputs=image_input, outputs=mid_output)

In [19]:
class CustomSig(tf.Module):
    def __init__(self,model):
        super(CustomSig, self).__init__()
        self.model=model

    @tf.function
    def __call__(self,image_tensor):
        return self.model(image_tensor)     

modul = CustomSig(final_model)

signatures = {"similarity": modul.__call__.get_concrete_function(tf.TensorSpec(shape=(None,224,224,3), dtype=tf.float32))}

tf.saved_model.save(modul, "custom_sig", signatures=signatures)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: custom_sig\assets


In [20]:
! saved_model_cli show --dir custom_sig --tag_set serve --signature_def similarity

The given SavedModel SignatureDef contains the following input(s):
  inputs['image_tensor'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 224, 224, 3)
      name: similarity_image_tensor:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['output_0'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 128)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [ ]:
! saved_model_cli run --dir custom_sig --tag_set serve --signature_def similarity --inputs "image_tensor=img.npy" --outdir model_cli_run_output

### Saving a model with a given output name and multiple Outputs

In [1]:
from tensorflow.keras.models import load_model
import tensorflow as tf
model = load_model('final_model.h5')

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
mid_output = model.get_layer('dense_7').output
image_input = model.get_layer('input_3').output
final_model = Model(inputs=image_input, outputs=mid_output)

In [5]:
class CustomSig(tf.Module):
    def __init__(self,model,final_model):
        super(CustomSig, self).__init__()
        self.model=model
        self.final_model=final_model

    @tf.function
    def __call__(self,image_tensor):
        return {'output_tensor': self.model(image_tensor),
                'intermediate_output':self.final_model(image_tensor)}     

modul = CustomSig(model, final_model)

signatures = {"similarity": modul.__call__.get_concrete_function(tf.TensorSpec(shape=(None,224,224,3), dtype=tf.float32))}

tf.saved_model.save(modul, "custom_sig", signatures=signatures)

INFO:tensorflow:Assets written to: custom_sig\assets


In [6]:
! saved_model_cli show --dir custom_sig --tag_set serve --signature_def similarity

The given SavedModel SignatureDef contains the following input(s):
  inputs['image_tensor'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 224, 224, 3)
      name: similarity_image_tensor:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['intermediate_output'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 128)
      name: StatefulPartitionedCall:0
  outputs['output_tensor'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:1
Method name is: tensorflow/serving/predict


### search for a file in a folder

In [ ]:
import os

def find_files(filename, search_path):
   result = []

# Wlaking top-down from the root
   for root, dir, files in os.walk(search_path):
      if filename in files:
         result.append(os.path.join(root, filename))
   return result

print(find_files("smpl.htm","D:"))

In [ ]:
result_train=[]
for root, directory, files in os.walk(search_path):
    z = list(set(files).intersection(train_files))
    if(len(z)>0):
        for i in z:
            result_train.append(os.path.join(root,i))

## gRPC request

In [1]:
import grpc
import numpy
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [23]:
channel = grpc.insecure_channel('localhost:8500')
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
request = predict_pb2.PredictRequest()
request.model_spec.name = 'my_model'
request.model_spec.signature_name = 'similarity'

In [24]:
final_image=[]

In [25]:
image_path ='bengal_cat.jpg'

image = load_img(image_path, target_size=(128, 128))
	# convert to array
image = img_to_array(image)
# reshape into a single sample with 3 channels
image = image.reshape(1, 128, 128, 3)
# center pixel data
image = image.astype('float32')
#image = image - [123.68, 116.779, 103.939]
final_image.extend(image)

In [26]:
image_path ='golden-retriever-royalty-free-image-506756303-1560962726.jpg'

image = load_img(image_path, target_size=(128, 128))
	# convert to array
image = img_to_array(image)
# reshape into a single sample with 3 channels
image = image.reshape(1, 128, 128, 3)
# center pixel data
image = image.astype('float32')
#image = image - [123.68, 116.779, 103.939]
final_image.extend(image)

In [27]:
final_image = numpy.array(final_image) 
type(final_image),type(image)

(numpy.ndarray, numpy.ndarray)

In [28]:
final_image.shape

(2, 128, 128, 3)

In [29]:
final_image.shape[0]

2

In [17]:
request.inputs['images'].CopyFrom(tf.make_tensor_proto(final_image, shape=[final_image.shape[0],128,128,3]))
result = stub.Predict(request, 10.0)

In [18]:
type(result)

tensorflow_serving.apis.predict_pb2.PredictResponse

In [19]:
floats = numpy.array(list(result.outputs['image_vectors'].float_val))
floats.shape

(400,)

In [20]:
image_float=numpy.split(floats,2)

In [21]:
image_float=numpy.array(image_float)

In [22]:
image_float.shape

(2, 200)

In [24]:
for i in floats:
    if(i>0.5):
        print('Dog ')
    else:
        print('Cat ')

Cat 
Dog 


## Any Shape Input Model

In [29]:

# first neural network with keras tutorial
from numpy import loadtxt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Reshape

In [30]:
x = Input(shape=(None,None,3))
y = Reshape((1,))(x)
model_any_shape_input = Model(x, y)

In [31]:
model_any_shape_input.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, None, None, 3)]   0         
_________________________________________________________________
reshape_5 (Reshape)          (None, 1)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [34]:
import tensorflow as tf
tf.saved_model.save(model_any_shape_input, "any_input")

INFO:tensorflow:Assets written to: any_input\assets


In [35]:
! saved_model_cli show --dir any_input --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['input_14'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, -1, -1, 3)
      name: serving_default_input_14:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['reshape_5'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: PartitionedCall:0
Method name is: tensorflow/serving/predict
2020-06-11 17:30:28.867483: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_101.dll


## Changing the depth of the image

In [37]:
img = np.array([[1, 2], [3, 4]])
print(img.shape)
print(img)
print()
stacked_img = np.stack((img,)*3, axis=-1)
print(stacked_img)
print(stacked_img.shape)

(2, 2)
[[1 2]
 [3 4]]

[[[1 1 1]
  [2 2 2]]

 [[3 3 3]
  [4 4 4]]]
(2, 2, 3)


In [44]:
print(img[0][1], stacked_img[0][1][0], stacked_img[0][1][1], stacked_img[0][1][2])

2 2 2 2


## Using REST Api

In [ ]:
import requests

url = "https://covid-193.p.rapidapi.com/statistics"

headers = {
    'x-rapidapi-host': "covid-193.p.rapidapi.com",
    'x-rapidapi-key': "ed8f7e8cfemshc66d9c7f282bcacp1925c8jsn0d873cee5415"
    }

response = requests.request("GET", url, headers=headers)

print(response.text)